## Imports, function definitions

Note: DeepMoD does not work for python3.9

In [1]:
# pip install pysindy

In [2]:
# pip install DeePyMoD

In [3]:
# pip install --upgrade tensorflow

In [4]:
# pip install tensorflow-io

In [5]:
results_path = ""

In [6]:
import matplotlib.pyplot as plt

# General imports
import numpy as np
import torch
import pandas as pd

# DeePyMoD imports
from deepymod import DeepMoD
from deepymod.data import Dataset, get_train_test_loader
from deepymod.data.samples import Subsample_random
from deepymod.data.burgers import burgers_delta
from deepymod.model.constraint import LeastSquares
from deepymod.model.func_approx import NN
from deepymod.model.library import Library1D
from deepymod.model.sparse_estimators import Threshold
from deepymod.training import train
from deepymod.training.sparsity_scheduler import Periodic, TrainTest, TrainTestPeriodic

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(device)

# Settings for reproducibility
np.random.seed(42)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


import pickle
from sklearn.metrics import mean_squared_error
def load_data(X,y):
    return torch.from_numpy(X).float(),torch.from_numpy(y).float()

def uxt_2D_to1D(u,x,t):
    '''
    Function for reshaping u,x and t
    '''
    t_reg=[]
    x_reg=[]
    for i in range(len(t)):
        for j in range(len(x)):
            t_reg.append(t[i])
            x_reg.append(x[j])
    u_reg = u.reshape((u.size, 1))

    return u_reg,x_reg,t_reg

descr = ['',
 'u_{x}',
 'u_{xx}',
 'u_{xxx}',
 'u',
 'uu_{x}',
 'uu_{xx}',
 'uu_{xxx}',
 'u^2',
 'u^2u_{x}',
 'u^2u_{xx}',
 'u^2u_{xxx}']

def return_pde(w, rhs_description=descr, ut = 'u_t'):
    pde = ut + ' = '
    first = True
    for i in range(len(w)):
        if w[i] != 0:
            if not first:
                pde = pde + ' + '
            pde = pde + "(%05f)" % (w[i].real) + rhs_description[i] 
            first = False
    return pde


cuda


In [7]:
def error_metric(true_coeff,pred_coeff):
    wrmse = mean_squared_error(true_coeff,pred_coeff,squared=False)
    return wrmse

## Testing 4.1

In [8]:
# Heat equation ut = 0.15uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.15

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("1_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(31,33):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))
    

SNR =  0.8
Dataset is using device:  cuda


2023-04-28 11:05:16.406641: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-28 11:05:16.928299: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


 19750  MSE: 1.09e+05  Reg: 5.02e-14  L1: 1.66e+07 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-1661032.000000)u_{x} + (-522322.062500)u_{xx} + (-325033.375000)u_{xxx} + (-7632.039062)uu_{x} + (-103582.265625)uu_{xx} + (13667.357422)uu_{xxx} + (7187.342773)u^2u_{x} + (8687.500977)u^2u_{xx} + (443.371338)u^2u_{xxx}  rmse =  512233.3575576538
Dataset is using device:  cuda
 19750  MSE: 9.20e+04  Reg: 1.82e-12  L1: 9.19e+06 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.055265) + (345596.312500)u_{x} + (139790.500000)u_{xx} + (57104.304688)u_{xxx} + (0.013875)u + (-45852.265625)uu_{x} + (12915.671875)uu_{xx} + (2448.599365)uu_{xxx} + (-0.000308)u^2 + (795.244019)u^2u_{x} + (-323.818329)u^2u_{xx} + (-74.271370)u^2u_{xxx}  rmse =  109740.2380054605
Dataset is using device:  cuda
 19750  MSE: 2.55e+04  Reg: 1.08e-10  L1: 4.40e+06 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (5.313342) + (-2106977.000000)u_{x} + (-188253.640625

KeyboardInterrupt: 

In [8]:
# Heat equation ut = 0.15uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.15

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("1_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(32,33):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))
    

SNR =  0.75
Dataset is using device:  cuda


2023-04-30 12:08:51.146304: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-30 12:08:51.784986: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


  2250  MSE: 1.15e+05  Reg: 2.60e-07  L1: 4.66e+05 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-36367.898438)u_{xx} + (4254.354980)uu_{xx}  rmse =  10570.14066230982
Dataset is using device:  cuda
  2250  MSE: 1.15e+05  Reg: 2.66e-08  L1: 3.44e+05 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.000001) + (-0.159184)u_{xx} + (19310.548828)u_{xxx} + (1076.343018)uu_{x} + (-715.074707)uu_{xxx} + (-133.433426)u^2u_{x} + (-208.392197)u^2u_{xxx}  rmse =  5587.399320896789
Dataset is using device:  cuda
  2500  MSE: 1.14e+05  Reg: 9.82e-07  L1: 6.12e+05 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-8993.789062)u_{x} + (3523.849609)u_{xx} + (176.915848)u_{xxx} + (2266.821533)uu_{x} + (-515.635132)uu_{xx} + (96.628670)uu_{xxx} + (-141.234665)u^2u_{x} + (13.362091)u^2u_{xx} + (-13.200691)u^2u_{xxx}  rmse =  2868.9428430716166
Dataset is using device:  cuda
  2250  MSE: 1.15e+05  Reg: 5.07e-05  L1: 3.59e+05 Algorithm converged. Wri

## Testing 4.2

In [9]:
# Heat equation ut = 0.15uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.15

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("1_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(33,35):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))
    

SNR =  0.7
Dataset is using device:  cuda
  2250  MSE: 1.13e+05  Reg: 3.92e-06  L1: 4.76e+05 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.000001) + (135005.000000)u_{x} + (-38049.542969)u_{xx} + (-38830.226562)uu_{x} + (3850.864014)uu_{xx} + (2542.854004)u^2u_{x}  rmse =  42034.91694913508
Dataset is using device:  cuda
  2500  MSE: 1.12e+05  Reg: 7.61e-06  L1: 5.92e+06 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (2676.218750)u_{xx} + (-122183.968750)u_{xxx} + (6634.831055)uu_{x} + (13956.289062)uu_{xx} + (7959.201172)uu_{xxx} + (-719.286865)u^2u_{x} + (-1554.104492)u^2u_{xx} + (595.736023)u^2u_{xxx}  rmse =  35638.835266750044
Dataset is using device:  cuda
  2250  MSE: 1.14e+05  Reg: 8.27e-06  L1: 2.48e+06 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-19239.121094)u_{x} + (-62180.406250)u_{xx} + (-44643.710938)u_{xxx} + (15170.120117)uu_{x} + (11362.708008)uu_{xx} + (8193.985352)uu_{xxx} + (-1396.453613)u^2u_{x} + (-

## Testing 4.3

In [10]:


# Heat equation ut = 0.15uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.15

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("1_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(35,37):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))
    

SNR =  0.5
Dataset is using device:  cuda
  1250  MSE: 1.18e+05  Reg: 3.83e-12  L1: 9.20e-01 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.619108)u_{xx}  rmse =  0.22202225831670777
Dataset is using device:  cuda
  2250  MSE: 1.18e+05  Reg: 2.29e-02  L1: 3.35e+06 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.000008) + (-168163.500000)u_{xx} + (317951.062500)u_{xxx} + (-907.024780)uu_{x} + (42914.789062)uu_{xx} + (-83512.750000)uu_{xxx} + (119.088638)u^2u_{x} + (-2730.142090)u^2u_{xx} + (5528.324219)u^2u_{xxx}  rmse =  107326.13955431862
Dataset is using device:  cuda
  2500  MSE: 1.18e+05  Reg: 1.10e-06  L1: 3.79e+06 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.000000) + (-101064.875000)u_{xx} + (98754.562500)u_{xxx} + (-18523.478516)uu_{x} + (5420.422363)uu_{xx} + (2757.596680)uu_{xxx} + (2049.801514)u^2u_{x} + (622.164551)u^2u_{xx} + (-1504.225586)u^2u_{xxx}  rmse =  41184.09779753904
Dataset is using device:  cu

## Testing 4.4

In [11]:


# Heat equation ut = 0.15uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.15

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("1_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(37,39):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))
    

SNR =  0.3
Dataset is using device:  cuda
  2250  MSE: 1.23e+05  Reg: 3.36e-06  L1: 1.24e+06 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.000001) + (-43230.761719)u_{x} + (35050.714844)u_{xx} + (-8664.615234)u_{xxx} + (2640.381836)uu_{x} + (-13117.863281)uu_{xx} + (804.604126)uu_{xxx} + (181.457001)u^2u_{x} + (975.685913)u^2u_{xx} + (7.810129)u^2u_{xxx}  rmse =  16716.256318216878
Dataset is using device:  cuda
 11000  MSE: 9.70e+04  Reg: 4.16e-13  L1: 1.11e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.000000) + (-0.343783)u_{xx}  rmse =  0.14254293165926166
Dataset is using device:  cuda
  2250  MSE: 1.28e+05  Reg: 8.57e-06  L1: 6.51e+06 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (21391.064453)u_{x} + (-852008.000000)u_{xx} + (-64866.203125)u_{xxx} + (66045.781250)uu_{x} + (42451.011719)uu_{xx} + (-2939.913086)uu_{xxx} + (-8585.097656)u^2u_{x} + (8015.064453)u^2u_{xx} + (1378.742188)u^2u_{xxx}  rmse =  247806.3

## Testing 4.5

In [8]:


# Heat equation ut = 0.15uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.15

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("1_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(39,41):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))
    

SNR =  0.2
Dataset is using device:  cuda


2023-05-05 18:04:21.195103: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-05 18:04:21.783394: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


  2250  MSE: 1.36e+05  Reg: 7.24e-04  L1: 3.60e+05 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.000001) + (-155662.062500)u_{x} + (-1.189284)u_{xx} + (1825.028564)u_{xxx} + (36086.527344)uu_{x} + (-224.937790)uu_{xxx} + (-2089.837158)u^2u_{x}  rmse =  46134.462809719786
Dataset is using device:  cuda
  1750  MSE: 1.34e+05  Reg: 5.52e-06  L1: 7.36e+05 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.000003) + (-349828.531250)u_{x} + (-0.273446)u_{xx} + (11446.826172)u_{xxx} + (40071.453125)uu_{x} + (-1311.163696)uu_{xxx}  rmse =  101701.55341812148
Dataset is using device:  cuda
 19750  MSE: 1.35e+05  Reg: 1.53e-07  L1: 6.92e+04 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.053005)u_{xx} + (26636.480469)u_{xxx} + (11010.714844)uu_{x} + (-6770.965332)uu_{xxx} + (-1195.012695)u^2u_{x} + (421.436218)u^2u_{xxx}  rmse =  8554.6749817925
Dataset is using device:  cuda
  2250  MSE: 1.33e+05  Reg: 2.68e-07  L1: 1.66e+06 Algori